# ZapMeNot Spherical Source Sensitivity Analysis

The objective is to evaluate the accuracy of the spherical source quadrature for both on-axis and off-axis shielding configurations.  Tests are run with quadratures ranging from 1 to 30 and a source of 1 MeV photons in a 10 cm radius void sphere.  The detector is 20 cm from the center of the sphere.

For this test "on-axis" means the detector is aligned with the z-axis of the sphere.  "off-axis" means the detector is alignment is orthogonal with the z-axis.

In [64]:
import sys
sys.path.append('/Users/alan/Desktop/github/ZapMeNot')

In [65]:
from zapmenot import model, source, shield, detector, material
import math
import numpy as np


### On-Axis Model

In [66]:
def test_sphere(nodes):
    myModel = model.Model()
    mySource = source.SphereSource("air", sphere_radius=10,
                                   sphere_center=[4, 5, 6], density=0)
    mySource.points_per_dimension = [nodes, nodes, nodes]
    photonEnergy = 1.0  # MeV
    photonIntensity = 1  # photons/sec
    mySource.add_photon(photonEnergy, photonIntensity)
    myModel.add_source(mySource)
    myModel.add_detector(detector.Detector(4, 5, 26))
    return myModel.calculate_exposure()

In [67]:
onAxisResults = []
for i in range(1, 30):
    onAxisResults.append(test_sphere(i))

### Off-Axis Model

In [68]:
def test_sphere_side(nodes):
    myModel = model.Model()
    mySource = source.SphereSource("air", sphere_radius=10,
                                   sphere_center=[4, 5, 6], density=0)
    mySource.points_per_dimension = [nodes, nodes, nodes]
    photonEnergy = 1.0  # MeV
    photonIntensity = 1  # photons/sec
    mySource.add_photon(photonEnergy, photonIntensity)
    myModel.add_source(mySource)
    myModel.add_detector(detector.Detector(24, 5, 6))
    return myModel.calculate_exposure()

In [69]:
offAxisResults = []
for i in range(1, 30):
    offAxisResults.append(test_sphere_side(i))

### Now for an analytical solution

In [70]:
R=10
a=10
volume = 4/3*math.pi*(R**3)
Sv = 1/volume # 1 Bq total
uncollidedFlux = Sv/4/(a+R)
uncollidedFlux = uncollidedFlux * (2*R*(a+R) - a*(a+2*R)*math.log((a+2*R)/a))

In [71]:
airMassEnAbsCoeff = 2.787E-02 # @ 1MeV (ANS 6.4.3 Table 2)
uncollidedEnergyFlux = uncollidedFlux * 1; # 1 MeV
uncollidedExposure = uncollidedEnergyFlux * 1.835E-8 * airMassEnAbsCoeff * 1000 * 3600  # mR/hr;

In [72]:
print("uncollided flux: ", uncollidedFlux, "photons/cm2/sec")
print("uncollided energy flux: ", uncollidedEnergyFlux, "MeV/cm2/sec")
print("uncollided exposure: ", uncollidedExposure, "mR/hr")


uncollided flux:  0.00021013320659717179 photons/cm2/sec
uncollided energy flux:  0.00021013320659717179 MeV/cm2/sec
uncollided exposure:  3.8687460762704145e-07 mR/hr


3.868745387518610e-07 mR/hr is the result provided by Matlab using a (10,10,10) quadrature.  Hence a (10, 10, 10) "on-axis" quadrature appears to be converged and validates the analytical solution.

Next, determine the relative error in on-axis and off-axis results as a funtion of quadrature.

In [73]:
onAxisError = (np.array(onAxisResults) - uncollidedExposure)/uncollidedExposure

In [74]:
offAxisError = (np.array(offAxisResults) - uncollidedExposure)/uncollidedExposure

In [75]:
onAxisError

array([-1.69972329e-01, -3.38506085e-02, -6.89694036e-03, -1.44861939e-03,
       -3.11579803e-04, -6.82763652e-05, -1.51847162e-05, -3.41776007e-06,
       -7.76857434e-07, -1.78029727e-07, -4.10810876e-08, -9.53572802e-09,
       -2.22474250e-09, -5.21357244e-10, -1.22654874e-10, -2.89554196e-11,
       -6.85563711e-12, -1.62633228e-12, -3.86433517e-13, -9.26400463e-14,
       -2.32626409e-14, -4.78936724e-15,  0.00000000e+00,  1.50522970e-15,
       -2.46310315e-15, -1.09471251e-15, -1.77890783e-15,  3.42097660e-15,
       -1.36839064e-16])

In [76]:
offAxisError

array([1.42368080e+00, 1.77809360e-01, 8.52373957e-02, 2.79065762e-02,
       1.17090690e-02, 4.68433966e-03, 1.99446732e-03, 8.51185956e-04,
       3.71503340e-04, 1.63527776e-04, 7.28239248e-05, 3.26811964e-05,
       1.47791105e-05, 6.72481813e-06, 3.07734769e-06, 1.41516794e-06,
       6.53690007e-07, 3.03152535e-07, 1.41096518e-07, 6.58856743e-08,
       3.08576289e-08, 1.44917782e-08, 6.82296582e-09, 3.21982851e-09,
       1.52273580e-09, 7.21576742e-10, 3.42568113e-10, 1.62918127e-10,
       7.76048542e-11])

The on-axis model appears to be closer to the analytical model for all quadrature levels.  Both models achieve an relative accuracy of 1e-4 by using at least a quadrature of 11.

In [77]:
# double the number of "Z" nodes
def test_sphere_side2(nodes):
    myModel = model.Model()
    mySource = source.SphereSource("air", sphere_radius=10,
                                   sphere_center=[4, 5, 6], density=0)
    mySource.points_per_dimension = [nodes, nodes, nodes*2]
    photonEnergy = 1.0  # MeV
    photonIntensity = 1  # photons/sec
    mySource.add_photon(photonEnergy, photonIntensity)
    myModel.add_source(mySource)
    myModel.add_detector(detector.Detector(24, 5, 6))
    return myModel.calculate_exposure()

In [78]:
offAxisResults = []
for i in range(1, 30):
    offAxisResults.append(test_sphere_side2(i))

In [79]:
offAxisError = (np.array(offAxisResults) - uncollidedExposure)/uncollidedExposure

In [80]:
offAxisError

array([ 4.62220647e-01,  1.90249079e-03,  8.49636735e-03,  2.43419301e-04,
        2.72973744e-04,  1.16838824e-05,  1.09496608e-05,  5.37127536e-07,
        4.90024118e-07,  2.53994629e-08,  2.33857134e-08,  1.24418100e-09,
        1.16447225e-09,  6.28631818e-11,  5.97778714e-11,  3.25936967e-12,
        3.14168807e-12,  1.73648772e-13,  1.68585727e-13,  9.30505635e-15,
        8.75770010e-15,  1.23155158e-15,  8.21034384e-16,  1.23155158e-15,
       -1.09471251e-15, -1.23155158e-15, -6.84195320e-16,  2.05258596e-15,
        6.84195320e-16])

These results are noticeably improved and are closer to the on-axis results.